<img width="10%" alt="Naas" src="https://landen.imgix.net/jtci2pxwjczr/assets/5ice39g4.png?w=160"/>

# OpenAI - Generate_Act_as_a_x_notebook
<a href="https://app.naas.ai/user-redirect/naas/downloader?url=https://raw.githubusercontent.com/jupyter-naas/awesome-notebooks/master/OpenAI/OpenAI_Generate_Act_as_a_x_notebook.ipynb" target="_parent"><img src="https://naasai-public.s3.eu-west-3.amazonaws.com/Open_in_Naas_Lab.svg"/></a><br><br><a href="https://bit.ly/3JyWIk6">Give Feedback</a> | <a href="https://github.com/jupyter-naas/awesome-notebooks/issues/new?assignees=&labels=bug&template=bug_report.md&title=OpenAI+-+Generate_Act_as_a_x_notebook:+Error+short+description">Bug report</a>

**Tags:** #openai #ai #machinelearning #deeplearning #notebooks #automation #gsheet

**Author:** [Florent Ravenel](https://www.linkedin.com/in/florent-ravenel/)

**Last update:** 2023-06-14 (Created: 2023-06-14)

**Description:** This notebook creates "Act as a ..." notebooks from a Google Sheets spreadsheet using OpenAI_Act_as_a_chef.ipynb as template.

**References:**
- [OpenAI Documentation](https://openai.com/docs/)
- [Awesome ChatGPT Prompts](https://github.com/f/awesome-chatgpt-prompts#act-as-a-chef)

## Input

### Import libraries

In [26]:
from papermill.iorw import (
    load_notebook_node,
    write_ipynb,
)
import naas
from naas_drivers import gsheet
import copy
import json
import subprocess
import re

### Setup variables

In [17]:
# Inputs
spreadsheet_url = "https://docs.google.com/spreadsheets/d/18HChEH5ezs8p8pi1Y-u9yQFZdBPtMlONNjrHiH0VT3U/edit#gid=205410028"
sheet_name = "AI for Work - Prompts"
notebook_init = "AI_for_Work_Create_chat_plugin.ipynb"
plugin_name_init = "AI for work"
prompt_init = "Prompt"

## Model

### Get data from Google Sheets spreadsheet

In [18]:
df_input = gsheet.connect(spreadsheet_url).get(sheet_name=sheet_name)
df_input = df_input[df_input["Naas"] == "W44-2023"].reset_index(drop=True)
print("Row fetched:", len(df_input))
df_input.head(1)

Row fetched: 182


,prompt_title,Slug,Collection ID,Item ID,Created On,Updated On,Published On,emoji,prompt_engineer,gpt_model,...,prompt_type,role,department,document,Example in Image Format,Bookmark,Like,prompt,Featured,Naas
0,📝 Create A Blog Post,chatgpt-prompt-b2b-content-marketing-manager-m...,64a58200e016a9897bc5ddfc,64a58200e016a9897bc5e414,Mon Jun 19 2023 08:25:46 GMT+0000 (Coordinated...,Tue Oct 31 2023 02:48:29 GMT+0000 (Coordinated...,Tue Oct 31 2023 02:59:27 GMT+0000 (Coordinated...,📝,"<a href=""https://www.linkedin.com/in/aiforwork...",GPT-4,...,Task + Evaluation,b2b-content-marketing-manager,marketing,Blog Post,None,None,None,"{""prompt"":""Develop a tailored Blog Post aligne...",False,W44-2023


### Load template notebook

In [21]:
# Load notebook
nb_init = load_notebook_node(notebook_init)

# Get variables to be replaced
variables_init = nb_init.cells[11].source
plugin_init = nb_init.cells[17].outputs[0]["text"]

print("Variables:\n", variables_init, "\n")
print("Plugin:\n", plugin_init)

Variables:
 # Mandatory
plugin_name = "AI for work"
prompt = "Prompt"

# Optional
avatar = ""
model = "gpt-4"
temperature = 0.5
output_path = None 

Plugin:
 {"name": "AI for work", "model": "gpt-4", "temperature": 0.5, "max_tokens": 8192, "prompt": "Prompt", "commands": [], "description": "", "avatar": ""}



## Output

### Create notebooks

In [42]:
def remove_emojis(text):
    emoji_pattern = re.compile("["
                           u"\U0001F600-\U0001F64F"  # emoticons
                           u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                           u"\U0001F680-\U0001F6FF"  # transport & map symbols
                           u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                           u"\U00002500-\U00002BEF"  # chinese char
                           u"\U00002702-\U000027B0"
                           u"\U00002702-\U000027B0"
                           u"\U000024C2-\U0001F251"
                           u"\U0001f926-\U0001f937"
                           u"\U00010000-\U0010ffff"
                           u"\u2640-\u2642"
                           u"\u2600-\u2B55"
                           u"\u200d"
                           u"\u23cf"
                           u"\u23e9"
                           u"\u231a"
                           u"\ufe0f"  # dingbats
                           u"\u3030"
                           "]+", flags=re.UNICODE)
    return emoji_pattern.sub(r'', text).strip()

for row in df_input.itertuples():
    # Init
    nb = copy.deepcopy(nb_init)
    prompt_title = row.prompt_title
    prompt = row.prompt.replace('"', "'")
    print("Started with:", prompt_title)
    
    # Prep data
    prompt_title_without_emojis = remove_emojis(prompt_title)
    title = f"# AI for Work - {prompt_title_without_emojis}"
    notebook_output = f"AI_for_Work_{prompt_title_without_emojis.replace(' ', '_')}.ipynb"
    variables = variables_init.replace(plugin_name_init, prompt_title_without_emojis).replace(prompt_init, prompt)
#     plugin_json = json.dumps(json.loads(plugin_init.replace(fullname_init, character).replace(avatar_init, avatar)))
    
    # Update notebook
    nb.cells[1].source = title
#     nb.cells[5].source = nb_description
#     nb.cells[6].source = references
    nb.cells[11].source = variables
#     nb.cells[19].outputs[0]["text"] = plugin_json
    
    # Save new notebook
    write_ipynb(nb, notebook_output)
    print("💾 Notebook saved:", notebook_output)
    break

Started with: 📝 Create A Blog Post
💾 Notebook saved: AI_for_Work_Create_A_Blog_Post.ipynb
